In [43]:
import pandas as pd
from ast import literal_eval

In [22]:
annotators_df = pd.read_csv('..\\annotator_labels\paragraph_gold_annotator.csv', index_col=False)
llama_df = pd.read_csv('..\ollama\llama3-instruct-8b-q4\llama3_prediction.csv', index_col=False)

In [26]:
annotators_df.head()

,text-id,text,label,version,annotator_paragraph
0,medium-17,"Due to the overreach on the initial layoffs, c...",1,3,[1 1 1 1 1 1 1 0 1]
1,medium-17,I used to be in Pharmacy. Then I did a researc...,0,3,[1 0 0 0 1 1 0 0 0]
2,medium-17,So if a medical center has any that are partia...,1,3,[0 0 0 0 0 0 1 0 0]
3,medium-17,Open vials are supposed to be discarded after ...,1,3,[1 0 0 1 1 0 1 0 0]
4,medium-17,It costs about 15-35 cents to make a mL of ins...,1,3,[1 0 0 0 0 1 0 1 1]


In [10]:
llama_df.head()

,text-id,text,label,version,llama3_labels,not_generated_properly
0,medium-17,"Due to the overreach on the initial layoffs, c...",1,3,"[0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1]",False
1,medium-17,I used to be in Pharmacy. Then I did a researc...,0,3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",False
2,medium-17,So if a medical center has any that are partia...,1,3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",False
3,medium-17,Open vials are supposed to be discarded after ...,1,3,"[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0]",False
4,medium-17,It costs about 15-35 cents to make a mL of ins...,1,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",False


In [50]:
labels = annotators_df['label']
annotators = annotators_df['annotator_paragraph']
llama = llama_df['llama3_labels'].apply(literal_eval)

In [51]:
# remove the last text
annotators = annotators[:164]
llama = llama[:164]
labels = labels[:164]

In [52]:
llama[0]

[0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1]

In [36]:
for i in range(len(labels)):
    

0      1
1      0
2      1
3      1
4      1
      ..
164    0
165    1
166    1
167    0
168    0
Name: label, Length: 169, dtype: int64